# Algoritmo final: Decision Tree

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import datetime

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('../../set_datos_sup_desc.csv')
set_pruebas = pd.read_csv("../properati_dataset_modificado_descripciones.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [3]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades.loc[:, 'Year'] = propiedades.loc[:,'created_on'].apply(lambda x: int(x.split('-')[0]))

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

# Columnas

In [4]:
col = ['superficie','lat', 'lon','property_type_encoded']

comodidades = ['seguridad','gimnasio', 'aire', 'pileta', 'cochera','transporte','cocina','comedor',\
'living', 'banos', 'balcon','ambientes','departamento', 'dormitorios','quincho', 'granito','porcelanato',\
'completo', 'excelente', 'edificio','lavadero', 'amplio', 'planta', 'cuadras', 'placard', 'terraza', 'acceso',\
'parrilla', 'luminoso','madera', 'credito', 'calidad', 'cucicba', 'amenities','expensas','mesada',\
'responsable','muebles', 'espacio', 'diario', 'independiente', 'cubierta','contrafrente', 'integrada',\
'recepcion','categoria', 'profesional', 'baulera', 'natural','unidades', 'principal', 'espacio', 'estacion', 'vestidor',\
'solarium', 'central','parquet', 'estrenar', 'granito','patio',  'casa', 'suite', 'vista', 'hall', 'agua',  \
'lote', 'fondo', 'acero', 'torre',  'horno',  'sala', 'salon', 'frente','toilette','piso', 'pisos', 'apto',\
'sobre','doble','marca','nuevo', 'bacha', 'ideal','excelente','auto',  'autos',  'ninos', 'sauna', 'lindo','rio',\
'gym', 'luz', 'sol', 'spa','laundry', 'monoambiente', 'ascensores', 'plantas', 'diseno', 'inoxidable', 'cubierto', \
'griferia','terminaciones', 'avenida', 'galeria', 'privado', 'placares', 'negocios', 'duplex', 'lavarropas', \
'ceramicos', 'reciclado', 'balcones', 'gas','ducha']

# Parametros del estimador

In [7]:
criterion = 'mse'
max_depth = 100
max_features = 0.8

# Con un set de validacion busco las columnas mas importantes

In [6]:
columnas = col + comodidades
X_train, X_test, y_train, y_test = train_test_split(propiedades.loc[:,columnas], \
                                        propiedades.loc[:,'price_aprox_usd'], test_size=0.2)
X_train = pd.DataFrame(X_train).reset_index(drop = True)
X_test = pd.DataFrame(X_test).reset_index(drop = True)

In [7]:
tree = DecisionTreeRegressor(criterion = criterion, max_depth = max_depth, max_features = max_features)
    
tree.fit(X_train,y_train)
precision = tree.score(X_test,y_test) * 100
error = mean_squared_error(tree.predict(X_test), y_test)
print("Precision = {:.2f} %,  Error = {:.0f}".format(precision,error))

Precision = 94.16 %,  Error = 2249959698


In [8]:
imp = tree.feature_importances_
print(max(imp),min(imp))
print(len(columnas))

len([x for x in imp if x >= 2e-3])

0.386967442934 7.29235311653e-05
119


42

In [9]:
c = [y for x,y in enumerate(columnas) if imp[x] >= 2e-3]
print(c)

['superficie', 'lat', 'lon', 'property_type_encoded', 'seguridad', 'gimnasio', 'pileta', 'cochera', 'cocina', 'comedor', 'living', 'banos', 'departamento', 'dormitorios', 'completo', 'edificio', 'amplio', 'planta', 'terraza', 'parrilla', 'amenities', 'responsable', 'categoria', 'profesional', 'casa', 'suite', 'vista', 'hall', 'torre', 'salon', 'toilette', 'piso', 'doble', 'excelente', 'sauna', 'rio', 'spa', 'ascensores', 'diseno', 'avenida', 'duplex', 'ducha']


# Con el set de pruebas, usando las 50 columnas mas importantes

In [10]:
columnas = c

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)

tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/decisionTree_importanciaColumnas.csv', index = False)

# Con un set de validacion busco mas columnas mas importantes

In [11]:
columnas = col + comodidades
X_train, X_test, y_train, y_test = train_test_split(propiedades.loc[:,columnas], \
                                        propiedades.loc[:,'price_aprox_usd'], test_size=0.2)
X_train = pd.DataFrame(X_train).reset_index(drop = True)
X_test = pd.DataFrame(X_test).reset_index(drop = True)

In [12]:
tree = DecisionTreeRegressor(criterion = criterion, max_depth = max_depth, max_features = max_features)
    
tree.fit(X_train,y_train)
precision = tree.score(X_test,y_test) * 100
error = mean_squared_error(tree.predict(X_test), y_test)
print("Precision = {:.2f} %,  Error = {:.0f}".format(precision,error))

Precision = 94.19 %,  Error = 2240074463


In [14]:
imp = tree.feature_importances_
print(max(imp),min(imp))
print(len(columnas))

len([x for x in imp if x >= 1e-3])

0.413244922935 2.23475232024e-05
119


79

In [15]:
c = [y for x,y in enumerate(columnas) if imp[x] >= 1e-3]
print(c)

['superficie', 'lat', 'lon', 'property_type_encoded', 'seguridad', 'gimnasio', 'aire', 'pileta', 'cochera', 'transporte', 'cocina', 'comedor', 'living', 'banos', 'balcon', 'ambientes', 'departamento', 'dormitorios', 'quincho', 'porcelanato', 'completo', 'excelente', 'edificio', 'lavadero', 'amplio', 'planta', 'cuadras', 'placard', 'terraza', 'acceso', 'parrilla', 'madera', 'amenities', 'responsable', 'muebles', 'diario', 'independiente', 'cubierta', 'contrafrente', 'recepcion', 'categoria', 'profesional', 'principal', 'solarium', 'central', 'estrenar', 'patio', 'casa', 'suite', 'vista', 'hall', 'lote', 'fondo', 'acero', 'torre', 'horno', 'sala', 'salon', 'frente', 'toilette', 'piso', 'pisos', 'apto', 'sobre', 'doble', 'nuevo', 'bacha', 'ideal', 'excelente', 'sauna', 'rio', 'sol', 'spa', 'plantas', 'diseno', 'griferia', 'terminaciones', 'avenida', 'ducha']


# Con el set de pruebas, usando las 80 columnas mas importantes

In [16]:
columnas = c

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)

tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/decisionTree_importancia80Columnas.csv', index = False)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(propiedades.loc[:,columnas], \
                                        propiedades.loc[:,'price_aprox_usd'], test_size=0.2)
tree = DecisionTreeRegressor(criterion = criterion, max_depth = max_depth, max_features = max_features) 
tree.fit(X_train,y_train)
precision = tree.score(X_test,y_test) * 100
error = mean_squared_error(tree.predict(X_test), y_test)
print("Precision = {:.2f} %,  Error = {:.0f}".format(precision,error))

Precision = 94.25 %,  Error = 2153461948


# Usando las columnas importantes para mi

In [4]:
columnas = ['superficie','lat', 'lon','property_type_encoded', \
'seguridad','gimnasio', 'aire', 'pileta', 'cochera','transporte','cocina','comedor',\
'living', 'banos', 'balcon','ambientes','departamento', 'dormitorios','quincho', 'granito','porcelanato',\
'completo', 'edificio','lavadero', 'amplio', 'terraza', 'acceso','parrilla', 'luminoso','madera',  'amenities',\
'mesada','muebles', 'espacio',  'cubierta','contrafrente', 'recepcion','categoria', 'profesional', 'baulera', \
'unidades', 'principal', 'estacion', 'vestidor', 'solarium', 'central', 'parquet','patio',  'casa', 'suite', 'vista',\
'hall','lote', 'fondo', 'acero', 'torre',  'horno',  'sala', 'salon', 'frente','toilette', 'bacha','auto',  'autos',\
'ninos', 'sauna','rio', 'gym', 'luz', 'sol', 'spa','laundry', 'monoambiente', 'ascensores', 'plantas',  'inoxidable', \
'cubierto','griferia','avenida', 'galeria', 'negocios', 'duplex', 'lavarropas', 'ceramicos', 'balcones', 'gas','ducha']

len(columnas)

87

In [5]:
set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)

tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/decisionTree_columnasSeleccionadas.csv', index = False)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(propiedades.loc[:,columnas], \
                                        propiedades.loc[:,'price_aprox_usd'], test_size=0.2)
tree = DecisionTreeRegressor(criterion = criterion, max_depth = max_depth, max_features = max_features) 
tree.fit(X_train,y_train)
precision = tree.score(X_test,y_test) * 100
error = mean_squared_error(tree.predict(X_test), y_test)
print("Precision = {:.2f} %,  Error = {:.0f}".format(precision,error))

Precision = 95.07 %,  Error = 1864764517
